# Fishvideo processing

TODO: add existing processing on the stereo video from the semester project

In [2]:
import cv2
from pathlib import Path

def cut_video_seconds(in_path: str, out_path: str, seconds: float, start_sec: float = 0.0):
    in_path = str(in_path)
    out_path = Path(out_path)
    out_path.parent.mkdir(parents=True, exist_ok=True)

    cap = cv2.VideoCapture(in_path)
    if not cap.isOpened():
        raise RuntimeError(f"Could not open video: {in_path}")

    fps = cap.get(cv2.CAP_PROP_FPS)
    if not fps or fps <= 0:
        fps = 30.0

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    start_frame = int(round(start_sec * fps))
    max_frames = int(round(seconds * fps))
    end_frame = min(start_frame + max_frames, total_frames)

    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    writer = cv2.VideoWriter(str(out_path), fourcc, fps, (w, h))
    if not writer.isOpened():
        cap.release()
        raise RuntimeError(f"Could not open VideoWriter: {out_path}")

    frames_written = 0
    try:
        while True:
            pos = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
            if pos >= end_frame:
                break
            ok, frame = cap.read()
            if not ok:
                break
            writer.write(frame)
            frames_written += 1
    finally:
        writer.release()
        cap.release()

    return str(out_path), frames_written, fps

if __name__ == "__main__":
    out, n, fps = cut_video_seconds(
        in_path="FishVideo_60sec.mp4",
        out_path="FishVideo_12sec.mp4",
        seconds=12,        # <-- X seconds here
        start_sec=30      # optional start time
    )
    print(f"Saved: {out} | frames: {n} | fps: {fps}")


Saved: FishVideo_12sec.mp4 | frames: 300 | fps: 25.0
